name: Eklavya Gupta
university: University of Petroleum and Energy Studies
course: B.Tech CSE AI ML   # currently in 5th semester
sapid: 500093960
university mail id: 500093960@stu.upes.ac.in
personal mail id: emessage.eg@gmail.com

loading data

In [1]:
# name: Eklavya Gupta
# university: University of Petroleum and Energy Studies
# course: B.Tech CSE AI ML   # currently in 5th semester
# sapid: 500093960
# university mail id: 500093960@stu.upes.ac.in
# personal mail id: emessage.eg@gmail.com
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.models import resnet18
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from PIL import Image


class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name, label = self.data[idx]
        image = read_image(img_name)
        
        # Convert the tensor to a PIL Image
        image = Image.fromarray(image.permute(1, 2, 0).numpy())
        
        if self.transform:
            image = self.transform(image)
        return image, label

data = pd.read_csv('dataset.csv')
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data = data.values


In [2]:
import torch
from torchvision.transforms import transforms
from PIL import Image
import numpy as np

# Define a transform to preprocess your input image
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_image_path = data[0][0]
input_image = Image.open(input_image_path)
input_tensor = transform(input_image)

model = resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, 3),
    nn.LogSoftmax(dim=1)
)


checkpoint_path = os.path.join(os.getcwd(), 'Models', 'model_epoch_33.pth')  # Specify the path to your saved checkpoint file
checkpoint = torch.load(checkpoint_path)

model.load_state_dict(checkpoint)

model.eval()


with torch.no_grad():
    output = model(input_tensor.unsqueeze(0))
    probabilities = torch.exp(output)
    print(probabilities)
    predicted_class = torch.argmax(probabilities)


predicted_class = predicted_class.item()
predicted_probabilities = probabilities.squeeze().numpy()

print(f"predicted class: {predicted_class}")
print(f"predicted probabilities: {predicted_probabilities}")


C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[0.0071, 0.9845, 0.0084]])
Predicted Class Index: 1
Predicted Probabilities: [0.00711263 0.98449194 0.00839544]
